In [1]:
import os
import sys

import json

import catboost
import pandas as pd
import numpy as np
import math
import trimesh
import matplotlib.cm as cm

from IPython.display import display, JSON

In [2]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 1000)

In [3]:
centers = {"10":[8.954200350109572,-9.847498392210483,19.567843901159694],"11":[13.09937968536542,-9.924633670174043,14.634967099817992],"12":[15.049623036566935,-10.226854922548371,7.791118236419994],"13":[17.017895054785498,-10.006336382185726,1.5282307402394357],"14":[18.83098040741727,-9.688896425950045,-5.840533026718971],"15":[21.193316589801945,-9.120580320366864,-14.18430980331734],"16":[21.138141921307685,-8.3356484624913,-22.143752495939857],"17":[22.34872295174148,-15.22699783111289,-23.57658936780863],"18":[20.797050222527346,-17.240441049431393,-14.066004323895283],"19":[17.49476236805186,-17.779927735839372,-4.474108293434313],"2":[-26.761061135560052,-7.931285420385467,-11.52619277290625],"20":[14.650215607700869,-17.964868079450103,3.0728760006913305],"21":[11.968996229458288,-18.004352811130673,9.251666205207043],"22":[7.839295726237228,-17.948887919930588,14.71387177392102],"23":[3.478571365574764,-16.38412998763924,17.46120829650739],"24":[-1.1073848609683998,-15.47367810242514,18.014274468034223],"25":[-5.894734384065381,-15.20579923300031,17.919415589429782],"26":[-10.226575314842039,-15.319723034306296,16.666337091407858],"27":[-15.17500562285825,-15.922294911462073,13.87809919235707],"28":[-18.518342758586655,-15.518783935781316,8.094775833407642],"29":[-20.631279508463564,-15.343961130375357,2.274389175474066],"3":[-24.055795377831323,-7.799101370778331,-3.4458822223235956],"30":[-22.823249608608506,-15.295728258927912,-5.163853018777627],"31":[-26.27226398616533,-14.996942896879252,-14.765778705164351],"4":[-21.870777511373234,-7.790015119708453,4.368617352534606],"5":[-19.72065673380848,-7.843992361945826,10.404364223555419],"6":[-16.712506578775567,-7.636773315250838,17.117850963122965],"7":[-11.731801334292394,-8.801114318277927,21.473033163721006],"8":[-5.17645624295829,-9.568343532280142,23.585354419022394],"9":[2.589747760353341,-9.915203501871684,22.78742915618382]}

In [4]:
centers = {"1": [28.79756175319794, 2.0087468546138005, -17.490909391530803], "2": [22.75894966749346, 1.9216960881286584, -8.283317044555242], "3": [15.61968277466411, 0.6062852430097933, 0.6676852431495782], "4": [8.433571190985042, 0.5459173987349046, 7.226045245637961], "6": [-2.918288367501936, 1.1335247277171085, 14.90534837891698], "7": [-10.88144905746791, 1.3788114578742852, 16.68777004198772], "8": [-19.40865892734102, 1.512507217611753, 14.904684833904371], "9": [-27.244588285743394, 1.5824311953576915, 9.758442384974364], "10": [-32.14743590386763, 1.9319359000081646, 2.8058613321725954], "11": [-34.8204540354096, 2.861210456517478, -5.240233174413174]}

In [5]:
centers = {"10":[11.641303267765187,4.252012583008929,17.15397156942749],"11":[17.054454127343924,3.9579468437384357,12.498427805524736],"12":[20.427630248197165,2.996471201920695,5.134317365709358],"13":[23.596950665228455,2.904867737864926,-1.723921118458056],"14":[26.057714429986664,3.479683496630562,-9.609066376368537],"15":[28.21525127517379,4.153419004858484,-18.427312343212513],"18":[25.85886768625229,-1.1914534238065175,-17.70472163362664],"19":[24.138244669756645,-2.548624169345491,-7.336757548067363],"2":[-26.53207032120986,4.286816630203792,-18.006296904420125],"20":[21.340514327089053,-3.0823566924504266,1.5151121215709547],"21":[18.139845073483414,-3.262263418730804,8.061354635566476],"22":[13.895859196179039,-2.7908441750884916,14.258909347080056],"23":[8.356362562114029,-1.818743424062085,17.15596030695389],"24":[3.088712836642797,-1.3646588929993777,18.72852190813604],"25":[-2.511084392274413,-1.2684118913940734,18.669695792773414],"26":[-7.834851246578757,-1.4018620269723643,17.086739329265836],"27":[-13.184785145750192,-2.3266846717379077,14.104137001136806],"28":[-17.22791410721022,-2.803533143552484,8.139797225285688],"29":[-19.458092137172766,-2.76853761827582,1.1304246626529495],"3":[-24.020202571444592,3.512227771627468,-9.36021400965612],"30":[-22.509719714059663,-2.814982236558888,-7.429367725213798],"31":[-24.76702597540695,-1.7726646943844162,-17.755786357283736],"4":[-21.516405793357823,3.038298451941312,-0.5247112499098383],"5":[-19.204821414032747,3.512621859314243,6.097766801807982],"6":[-16.356603702661065,4.142262376252876,12.926113738072564],"7":[-10.529901764811246,4.362003415377335,17.43334963320893],"8":[-2.674741779398765,3.921220940182142,20.078883662905508],"9":[4.633733755500443,4.338141951909839,20.541880900435093]}

In [6]:
centers = {k: np.asarray(x) for k, x in centers.items()}

In [14]:
val_path = os.path.join('baseline', 'data', 'val.csv')
val_df = pd.read_csv(val_path)

In [31]:
def make_item(row, prefix='tooth'):
    item = {}

    for idx in range(1, 33):
        if f'{prefix}_{idx}_0' in row and not math.isnan(row[f'{prefix}_{idx}_0']):
            item[str(idx)] = [row[f'{prefix}_{idx}_{d}'] for d in range(3)]

    return item

In [65]:
row = val_df.sample().to_dict(orient='records')[0]

origin_roots = make_item(row, prefix='origin')
centers = make_item(row, prefix='center')

is_up = row['jaw'] == 0
gap = 16 if not is_up else 0

origin_roots = {str(int(k) + gap): np.asarray(x)*row['norm'] for k, x in origin_roots.items()}
centers = {str(int(k) + gap): np.asarray(x)*row['norm'] for k, x in centers.items()}

In [66]:
# list(val_df)
is_up


False

In [67]:
def make_scene(axis=True, scale=20):
    ambient = trimesh.scene.lighting.Light(
        color=np.array([255, 255, 255]),
        intensity=1.0
    )

    s = trimesh.Scene(lights=[ambient])

    if axis:
        axis = trimesh.creation.axis()
        axis.apply_scale(scale)
        s.add_geometry(axis)

    return s


def plot_points(scene, points, radius=0.125, color=None, translation=None):
    colors = cm.rainbow(np.linspace(0, 1, len(points)*2)) * 255
    for i, pt in enumerate(points):
        if translation is not None:
            pt = pt + translation
        m = trimesh.primitives.Sphere(
            radius=max(abs(radius[i]) if hasattr(radius, "__len__") else radius, 0.01),
            center=pt
        )
        m.visual.vertex_colors = color if color is not None else colors[i if i % 2 == 1 else (len(points)*2 - 1)]
        scene.add_geometry(m)

In [68]:
import warnings
from catboost import CatBoostRegressor

from txparser.transforms import normal_jaw, vtk_rotate, apply_tr_dict
from txparser.utils import read_json

class RootsModel:

    def __init__(self, model_folder=None) -> None:
        if model_folder is None:
            model_folder = MODEL_DEFAULT_PATH
        self._models = self._load_models(model_folder)


    def _load_model(self, model_dir, model_file, idx, axis):
        '''
        Load model from file
        Args:
            model_dir: str, path to the model directory
            model_file: str, model file name
            idx: int, model index
            axis: int, model axis
        Returns:
            CatBoostRegressor, model
            dict, model info
        '''
        model = CatBoostRegressor()
        path = os.path.join(model_dir, str(idx) + '-' + str(axis) + '-' + model_file)

        model_info = read_json(path + '.json')
        model.load_model(path)

        return model, model_info


    def _load_models(self, model_dir, model_file='root.model', axis=None):
        '''
        Load all models from the directory
        Args:
            model_dir: str, path to the model directory
            model_file: str, model file name
            axis: list, model axis
        Returns:
            dict, models
        '''
        models = {}

        if axis is None:
            axis = [0, 1, 2]

        for idx in range(1, 17):
            models[idx] = {}
            for a in axis:
                models[idx][a] = self._load_model(model_dir, model_file, idx, a)
        return models


    def _cut(self, roots, jaw):
        return {k: v for k, v in roots.items() if str(k) in jaw and all(x is not None for x in v)}


    def _invert(self, predict, forward):
        backforward = [np.linalg.inv(x) for x in reversed(forward[1:])]

        roots = apply_tr_dict(predict, backforward)

        return apply_tr_dict(roots, [np.linalg.inv(forward[0])])


    def _jaw_data(self, centers):
        '''
        Get jaw data
        Args:
            centers: dict, both jaws centers (1-32)
        Returns:
            dict, jaw data
        '''
        result = {}

        for i, idx in enumerate(centers.keys()):

            result.update({
                f'center_{idx}_{d}': v for d, v in enumerate(centers[idx])
            })

        return result


    def _predict(self, features):
        models = self._models
        result = {}

        for i in range(1, 17):

            result[str(i)] = [
                models[i][0][0].predict(features),
                models[i][1][0].predict(features),
                models[i][2][0].predict(features)
            ]
        return result

    @staticmethod
    def _take_closest_root(k, center, centers, roots):
        if str(k) in roots:
            return roots[str(k)]

        options = [
            key_
            for key_ in centers
            if key_ != str(k) and
               key_ != k and
               key_ in roots.keys() and
               roots[key_] is not None
        ]

        if not options:
            print(f'No close root found for tooth {k} in {centers.keys()} centers and {roots.keys()} roots')
            return None

        closest_k = min(options, key=lambda x: np.linalg.norm(centers[x] - center))

        approx_root = roots[closest_k] + (center - centers[closest_k])

        return approx_root

    def predict_jaw(self, centers, is_upper=False):

        # tindices = [int(x) for x in centers.keys()]

        # jaw = {str(idx)   : centers[str(idx)] for idx in range(1, 17) if str(idx) in centers}

        norm_teeth, forward, _ = normal_jaw(centers, fit=True, rotate=False)
        z_axis = np.asarray([0, 0, 1])
        z_rot  = np.array(vtk_rotate(180, z_axis))

        features_names = self._models[1][0][1]['features']

        if not is_upper:
            norm_teeth = apply_tr_dict(norm_teeth, [z_rot])

        if len(norm_teeth) >= 4:
            features = self._jaw_data(norm_teeth)
            feat     = [0 if is_upper else 1] + [features[name] if name in features else None for name in features_names[1:]]
            predict  = self._predict(feat)
            if not is_upper:
                forward += [z_rot]
            roots    = self._invert(predict, forward)

        for k, v in centers.items():
            if k in roots or not roots:
                continue
            root = RootsModel._take_closest_root(k, v, centers, roots)
            if root is None:
                # print(f'No close root found for upper tooth {k} in {centers.keys()} centers and {up_roots.keys()} roots')
                continue
            roots[k] = root

        # print('roots:predict', centers.keys(), norm_teeth.keys(), norm_teeth.keys())

        roots    = self._cut(roots, centers)

        return roots


    def predict(self, centers):

        tindices = [int(x) for x in centers.keys()]

        if max(tindices) < 17 or min(tindices) > 16:
            warnings.warn('Prefer to use both jaws for roots prediction. do not forget make low jaw 17-32 indices!', UserWarning)

        centers = {str(k): np.asarray(x) for k, x in centers.items() if x is not None}

        norm_teeth, forward, _ = normal_jaw(centers, fit=True, rotate=False)
        z_axis = np.asarray([0, 0, 1])
        z_rot    = np.array(vtk_rotate(180, z_axis))

        up_jaw = {str(idx)   : norm_teeth[str(idx)] for idx in range(1, 17) if str(idx) in centers}
        lo_jaw = {str(idx-16): norm_teeth[str(idx)] for idx in range(17, 33) if str(idx) in centers}

        features_names = self._models[1][0][1]['features']
        up_roots, lo_roots = {}, {}

        if len(up_jaw) >= 4:
            up_features = self._jaw_data(up_jaw)
            up_feat     = [0] + [up_features[name] if name in up_features else None for name in features_names[1:]]
            up_predict  = self._predict(up_feat)
            up_roots    = self._invert(up_predict, forward)

        if len(lo_jaw) >= 4:
            lo_norm_teeth = apply_tr_dict(lo_jaw, [z_rot])
            lo_features = self._jaw_data(lo_norm_teeth)
            lo_feat     = [1] + [lo_features[name] if name in lo_features else None for name in features_names[1:]]
            lo_predict  = self._predict(lo_feat)
            lo_roots    = self._invert(lo_predict, forward + [z_rot])
            lo_roots    = {str(int(k) + 16): v for k, v in lo_roots.items() if v is not None}

        for k in range(1, 17):
            k = str(k)
            if k not in centers:
                continue
            if k in up_roots or not up_roots:
                continue
            root = RootsModel._take_closest_root(k, centers[k], centers, up_roots)
            if root is None:
                # print(f'No close root found for upper tooth {k} in {centers.keys()} centers and {up_roots.keys()} roots')
                continue
            # print(f'redefine upper root for tooth {k}')
            up_roots[k] = root

        for k in range(17, 33):
            k = str(k)
            if k not in centers:
                continue
            if k in lo_roots or not lo_roots:
                continue
            root = RootsModel._take_closest_root(k, centers[k], centers, lo_roots)
            if root is None:
                # print(f'No close root found for lower tooth {k} in {centers.keys()} centers and {lo_roots.keys()} roots')
                continue
            # print(f'redefine lower root for tooth {k}')
            lo_roots[k] = root

        up_roots    = self._cut(up_roots, centers)
        lo_roots    = self._cut(lo_roots, centers)

        roots = {
            **{str(k): v.tolist() for k, v in up_roots.items() if v is not None},
            **{str(k): v.tolist() for k, v in lo_roots.items() if v is not None},
        }

        return roots


In [69]:
centers

{'18': array([-26.51500994,  -1.05564744, -26.0970987 ]),
 '19': array([-23.95956971,  -0.17099421, -13.94895068]),
 '20': array([-20.87011606,   0.69464004,  -4.84712556]),
 '21': array([-18.56106045,   0.44125852,   2.26997881]),
 '22': array([-14.64429513,   0.26076696,   9.04389739]),
 '23': array([-9.47797141e+00, -9.37655718e-03,  1.42564253e+01]),
 '24': array([-3.19644451, -0.10752522, 16.2605863 ]),
 '25': array([ 2.84161052, -0.32801604, 15.93064411]),
 '26': array([ 8.98941137, -0.03335655, 14.86610878]),
 '27': array([15.10706473,  0.6695081 , 10.98161106]),
 '28': array([18.81894501,  0.32097498,  4.28148653]),
 '29': array([21.11715392,  0.28973072, -3.50044957]),
 '30': array([ 24.16065978,  -0.28397276, -13.51414965]),
 '31': array([ 26.18962188,  -0.68799054, -25.9829641 ])}

In [71]:
root_model = RootsModel('models')

In [72]:
roots = root_model.predict(centers)

/tmp/ipykernel_203490/1304624424.py:170: UserWarning: Prefer to use both jaws for roots prediction. do not forget make low jaw 17-32 indices!
  warnings.warn('Prefer to use both jaws for roots prediction. do not forget make low jaw 17-32 indices!', UserWarning)
/home/sigura/miniconda3/envs/ml2txparser/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [73]:
s = make_scene()

plot_points(s, centers.values(), radius=1)
plot_points(s, roots.values(), radius=0.5)
plot_points(s, origin_roots.values(), radius=0.75)

s.show(axes=1, height=500, line_settings={'point_size': 1})